<a href="https://colab.research.google.com/github/metsumesquita/ML-Python/blob/main/SLRTools_(template).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To Do

 - [ ] Criar classe para IEEE ou outra base de dados;
 - [ ] Considerar mais metadados. exemplo: tipo de artigo, ano;
 - [ ] Aplicação de mais filtros.



# Automação de Etapas de uma Revisão da Literatura
1) Criação de Strings de busca para diferentes bases acadêmicas;

2) Criação de conjunto de dados formado por metadados de estudos.

# Setup



In [ ]:
!pip install bibtexparser

In [ ]:
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from google.colab import drive
from datetime import date
from pprint import pprint
import pandas as pd
import bibtexparser
import requests
import os
import re

### Login no Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

# Constuindo Conjuntos de dados

## Classes

In [ ]:
@dataclass
class Artigo(ABC):
  entrada: dict
  editora: str = field(default="")
  string_generica: str = field(default="")

  def __post_init__(self) -> None:

    if "doi" in self.entrada:
      doi = self.entrada["doi"]
      prefix = "https://doi.org/"
      self.doi = doi if prefix in doi else f"{prefix}{doi}"
    else:
      self.doi = ""

    self.titulo = self.entrada["title"] if "title" in self.entrada else ""
    self.resumo = self.entrada["abstract"] if "abstract" in self.entrada else ""
    self.palavras_chave = self.entrada["keywords"] if "keywords" in self.entrada else ""
    self.editora = self.editora

  staticmethod
  @abstractmethod
  def gera_url() -> None:
    pass

  staticmethod
  @abstractmethod
  def cria_string(termos: list) -> tuple:

    if any([isinstance(termo, str) for termo in termos]):
      result = f" OR ".join([f"\"{termo}\"" if len(termo.split()) > 1 else termo for termo in termos])
      Artigo.string_generica = f"({result})" if len(termos) > 1 else result

    if any([isinstance(termo, list) for termo in termos]):
      result = f" AND ".join([Artigo.cria_string(termo) for termo in termos])
      Artigo.string_generica = f"({result})" if "(" in result else result

    return Artigo.string_generica


@dataclass
class ScienceDirectArtigo(Artigo):

  def __post_init__(self) -> None:
    super().__post_init__()
    self.capes_url = self.entrada["url"].replace(
        "https://www.sciencedirect.com",
        "https://www-sciencedirect.ez294.periodicos.capes.gov.br") if "url" in self.entrada else ""

  staticmethod
  def gera_url() -> None:
    pass

  staticmethod
  def cria_string(termos: list) -> tuple:
    return None


@dataclass
class IEEEArtigo(Artigo):

  def __post_init__(self) -> None:
    super().__post_init__()
    resposta = requests.get(self.doi)
    url = resposta.url
    self.capes_url = url.replace(
        "https://ieeexplore.ieee.org",
        "https://ieeexplore-ieee-org.ez294.periodicos.capes.gov.br") if url else ""

  staticmethod
  def gera_url(campos: list = ["All Metadata"]) -> None:
    pass


  staticmethod
  def cria_string(termos: list) -> tuple:
    return None

## Montando URLs e Strings de Busca

**Exemplos Entrada**:
```
[
      ["Fight", "Violence"],
      ["Detection", "Recognition"],
],
    ["Video Surveillance"],
    ["Machine Learning", "Deep Learning", "Reinforcement Learning", "Neural Network"],
],
```

**String formada:**

*(((Fight OR Violence) AND (Detection OR Recognition)) AND "Video Surveillance" AND ("Machine Learning" OR "Deep Learning" OR "Reinforcement Learning" OR "Neural Network"))*

**Exemplos Saída**:

URLs:
 - [Science Direct](https://www-sciencedirect.ez294.periodicos.capes.gov.br/search?qs=%28%28%28Fight%20OR%20Violence%29%20AND%20%28Detection%20OR%20Recognition%29%29%20AND%20%22Video%20Surveillance%22%20AND%20%28%22Machine%20Learning%22%20OR%20%22Deep%20Learning%22%20OR%20%22Reinforcement%20Learning%22%20OR%20%22Neural%20Network%22%29%29)

 - [IEEE](https://ieeexplore-ieee-org.ez294.periodicos.capes.gov.br/search/searchresult.jsp?action=search&matchBoolean=true&newsearch=true&queryText=(((%22All%20Metadata%22:%20Fight%20OR%20%22All%20Metadata%22:%20Violence)%20AND%20(%22All%20Metadata%22:%20Detection%20OR%20%22All%20Metadata%22:%20Recognition))%20AND%20%22All%20Metadata%22:%20%22Video%20Surveillance%22%20AND%20(%22All%20Metadata%22:%20%22Machine%20Learning%22%20OR%20%22All%20Metadata%22:%20%22Deep%20Learning%22%20OR%20%22All%20Metadata%22:%20%22Reinforcement%20Learning%22%20OR%20%22All%20Metadata%22:%20%22Neural%20Network%22)))



### Termos de Busca

In [ ]:
termos = [
  [
    [
      ["Fight", "Violence"],
      ["Detection", "Recognition"],
    ],
    ["Video Surveillance"],
    ["Machine Learning", "Deep Learning", "Reinforcement Learning", "Neural Network"],
  ],
]

### Montando URLs

In [ ]:
root_path = "/content/drive/MyDrive/Research/Curso/"
item = {"database": "", "url": "", "string_generica": "", "string_especifica": ""}

url_dataset = pd.DataFrame(columns=item.keys())

## Construindo Dataset de artigos

**Entrada:**

Localização de arquivos .bib baixados;

**Saída:**

Arquivo csv contendo os metadados dos estudos;

Ex. de colunas:
- doi;
- url;
- titulo;
- resumo;
- palavras-chave;
- editora.

## Filtrar conjunto de dados


Limpeza e filtragem de dados;

Pode conter:

 - Remoção de estudos duplicados;
 - Remoção de estudos não escritos em inglês;
 - Remoção de tipos específicos de artigos;
 - Normalização das informações;
 - Etc.

# Exportando Dataset

## Dataset com *strings* de busca

## Dataset Completo

## Filtrado e ajustado para o fast²